In [1]:
import json
import requests
import numpy as np
from requests.auth import HTTPBasicAuth
from sklearn.feature_extraction.text import TfidfVectorizer

# disable warnings related to verify
requests.urllib3.disable_warnings()

In [2]:
def gen_url(abstract_url, size=10):
    """
    Give finer control of the api
    :abstract_url str: base url
    :size int: number of documents to return
    """
    return "{}&size={}".format(abstract_url, size)

In [4]:
topics = None
with open('../topics/SP12022topics.json') as f_in:
    topics = json.load(f_in)

topic = topics[0]

topic_content_file = '../topics/topic_related_content/topic' + topic['topic_id'] + '.md'
topic_text = topic['topic_text']
abstract_url = topic['abstract_url']
resp = requests.get(gen_url(abstract_url, 100), auth=HTTPBasicAuth('inex', 'qatc2011'), verify=False)
contents = json.loads(resp.content)
abstracts = []
for hit in contents['hits']['hits']:
    if hit['_source']['abstract'] not in abstracts:
        abstracts.append(hit['_source']['abstract'])


In [5]:
def similarity_dot_prod(X):
    """
    Compute simularity using dot product
    :X: vectorized data, where
       X[0] = query vector
       X[1:] = data vectors
    """
    similarity = X @ X[0].T
    # sim(qeury, query) is not meaningful, remove it
    similarity = np.delete(similarity.toarray(), 0, 0)
    return similarity, np.argmax(similarity)

if abstracts[0] != topic_text:
    abstracts.insert(0, topic_text) # insert topic text

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(abstracts)

abstracts.pop(0) # remove topic_text
_, most_relevent_idx = similarity_dot_prod(X)

print("most relevent doc is at index {}".format(most_relevent_idx))
abstracts[most_relevent_idx]


most relevent doc is at index 51


'PDA’s (Personal Digital Assistant) and mobile phones have become multitasking devices that provide an enormous potential in various fields, including HACCP. We have designed a record taking software in order to be used in small food catering establishments, like canteens or restaurants. It’s developed for Windows Mobile® for use in mobile devices such as PDA’s and Smartphone’s. It enables the creation of four types of records: -Reception; -Temperature; -Cleaning; -Chlorine level in water. It’s possible to export data to common file formats like .pdf or .word. An Android OS version is currently under development. '